In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM

No dicts found; please check load_dicts...
Optimize a model with 314 rows, 314 columns and 314 nonzeros
Model has 49455 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e-03, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 314 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 314 columns, 0 nonzeros
Presolved model has 49455 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.54642552e+07  0.00000000e+00  2.25e+04 2.61e+02  9.27e+05     0s
   1   1.23018833e+07 -1.61590199e+07  2.78e+03 2.27e-13  2.10e+05     0s
   2   4.15559492e+06 -5.31733120e+06  2.78e-03 1.14e-13  3.02e+04     0s
   3   4.29

In [2]:
xi_list = GLS_Apr_weekday_PM.xi_list
P = GLS_Apr_weekday_PM.P
L = GLS_Apr_weekday_PM.L  # dimension of xi

314

In [3]:
size(P, 1), size(P, 2)

(56,314)

In [4]:
P

56×314 Array{Float64,2}:
 1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [5]:
xi_list

314-element Array{Any,1}:
 291.839 
  63.1725
  51.982 
 142.903 
 130.469 
  35.5594
 149.613 
  83.2788
  48.0737
  83.6789
  42.1837
 118.966 
  87.7583
   ⋮     
  82.0023
  85.0481
  41.8229
  33.087 
  34.9794
  29.4392
 205.952 
 105.947 
 111.109 
 214.739 
  30.4578
  29.7691

In [6]:
using JuMP

mGLSJulia = Model()

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    70022
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    52752

Total number of variables............................:    17640
                     variables with only lower bounds:    17640
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [7]:
getvalue(lam)

56-element Array{Float64,1}:
  406.993
  308.932
  406.828
  543.268
  524.971
  533.89 
  192.606
  501.848
  364.412
  432.304
  602.279
  670.196
  593.929
    ⋮    
  703.492
  526.87 
  799.146
 1687.19 
  174.456
  256.364
  237.224
  500.517
  398.022
  392.672
  562.337
  274.966

In [8]:
getobjectivevalue(mGLSJulia)

1.000000000000009

In [9]:
GLS_Apr_weekday_PM.saveDemandVec(getvalue(lam))

In [10]:
getvalue(p)

56×314 Array{Float64,2}:
 0.717061     0.155217     0.127722     …  0.0          0.0        
 1.86665e-15  1.86663e-15  1.86663e-15     0.0          0.0        
 1.73846e-16  1.73815e-16  1.73813e-16     0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0          …  0.0          0.0        
 2.48255e-15  2.48254e-15  2.48253e-15     1.05842e-15  1.05842e-15
 0.0          0.0          0.0             0.0          0.0        
 1.0391e-15   1.03907e-15  1.03907e-15     0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0          …  0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 ⋮                                      ⋱                          
 0.0          0.0      

In [11]:
route_choice_matrix_Apr_PM = getvalue(p)

56×314 Array{Float64,2}:
 0.717061     0.155217     0.127722     …  0.0          0.0        
 1.86665e-15  1.86663e-15  1.86663e-15     0.0          0.0        
 1.73846e-16  1.73815e-16  1.73813e-16     0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0          …  0.0          0.0        
 2.48255e-15  2.48254e-15  2.48253e-15     1.05842e-15  1.05842e-15
 0.0          0.0          0.0             0.0          0.0        
 1.0391e-15   1.03907e-15  1.03907e-15     0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0          …  0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 0.0          0.0          0.0             0.0          0.0        
 ⋮                                      ⋱                          
 0.0          0.0      

In [12]:
sum(route_choice_matrix_Apr_PM[1, :])

1.00000000000032

In [13]:
route_choice_matrix_dict_Apr_PM = Dict()

for row_idx = 1:size(route_choice_matrix_Apr_PM)[1]
    for col_idx = 1:size(route_choice_matrix_Apr_PM)[2]
        route_choice_matrix_dict_Apr_PM[(row_idx, col_idx)] = route_choice_matrix_Apr_PM[row_idx, col_idx]
    end
end

In [14]:
route_choice_matrix_dict_Apr_PM

Dict{Any,Any} with 17584 entries:
  (50,189) => 5.00283e-15
  (29,164) => 0.0
  (53,24)  => 2.99979e-15
  (47,153) => 0.0
  (4,173)  => 7.02814e-16
  (30,197) => 0.0
  (41,34)  => 0.0
  (8,160)  => 0.0
  (13,202) => 0.0
  (16,67)  => 5.08845e-15
  (4,296)  => 0.0
  (51,182) => 7.4538e-15
  (1,256)  => 6.92818e-15
  (39,219) => 0.1601
  (18,138) => 0.0
  (18,185) => 0.0
  (19,26)  => 0.0
  (50,27)  => 4.15453e-15
  (52,187) => 2.76166e-15
  (2,123)  => 0.0
  (25,243) => 2.69716e-15
  (35,22)  => 3.96862e-15
  (28,39)  => 2.95802e-15
  (35,142) => 1.80066e-15
  (12,145) => 0.0
  ⋮        => ⋮

In [15]:
outfile = open("../temp_files/route_choice_CDC_Apr_PM.json", "w")

JSON.print(outfile, route_choice_matrix_dict_Apr_PM)

close(outfile)